https://habr.com/ru/companies/hh/articles/303168/

https://github.com/hhru/api/blob/master/docs/vacancies.md

https://api.hh.ru/openapi/redoc#tag/Poisk-vakansij/operation/get-vacancies

In [ ]:
import requests
import json
from selenium import webdriver
from selenium.webdriver.common.by import By

In [ ]:
# Загружаем существующие данные
try:
    with open('vacancies.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
except FileNotFoundError:
    data = []

# Собираем существующие id
existing_ids = {item['id'] for item in data}

In [139]:
# Пустой json для объединения всех страниц
vacancies_list = {'items': []}

url = 'https://api.hh.ru/vacancies?'
params_dict = {
    'text': 'промт',
    'per_page': '100',
    'schedule': 'fullDay',
    'work_format': 'REMOTE'
}

query_params = '&'.join(f"{key}={value}" for key, value in params_dict.items())
query = url + query_params

headers = {'User-Agent': 'api-test-agent'}

# Отключаем проверку SSL-сертификата (аналог флага -k в curl)
response = requests.get(query, headers=headers, verify=False)

vacancies_list = response.json()


pages = response.json()['pages']

print(f"Всего: {pages} страниц")
print("Страница 0 обработана")

if pages > 1:
    for i in range(pages):
        if i == 0:
            None
        else:
            global vacancies_list
            query = query + f"&page={i}"
            response = requests.get(query, headers=headers, verify=False)
            vacancies_list = {'items': vacancies_list['items'] + response.json()['items']}
            query = query.replace(f"&page={i}", "")
            print(f"Страница {i} обработана")
else:
    None

c:\Users\alexe\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.hh.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Всего: 4 страниц
Страница 0 обработана


c:\Users\alexe\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.hh.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Страница 1 обработана


c:\Users\alexe\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.hh.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Страница 2 обработана


c:\Users\alexe\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.hh.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Страница 3 обработана


In [142]:
# Открываем браузер
driver = webdriver.Chrome()

# Добавляем новые
for item in vacancies_list['items']:
    if item['id'] not in existing_ids:
        employer = item.get('employer', {})
        accredited_it_employer = employer.get('accredited_it_employer')
        if accredited_it_employer:
            IT = 'Да'
        else:
            IT = 'Нет'

        salary = item.get('salary', {})
        if salary != None:
            salary_from = salary.get('from')
            salary_to = salary.get('to')
            salary_cur = salary.get('currency')
        else:
            salary_from = None
            salary_to = None
            salary_cur = None

        snippet = item.get('snippet', {})

        # Переходим на страницу
        vacancy_url = item['alternate_url']
        try:
            driver.get(vacancy_url)
            # Находим элемент по атрибуту data-qa
            element = driver.find_element(By.XPATH, 
                "//*[@data-qa='vacancy-description']")
            vacancy_description = element.text
        except Exception as e:
            vacancy_description = f"Ошибка: {e}"
        
        data.append({
            'id': item['id'],
            'Название': item['name'],
            'От': salary_from,
            'До': salary_to,
            'Валюта': salary_cur,
            'Описание': vacancy_description,
            # 'Кратко': abstract,
            # 'Требования': snippet.get('requirement'),
            # 'Ответственность': snippet.get('responsibility'),
            'Компания': employer.get('name'),
            'IT': IT,
            'Опубликовано': item['published_at'],
            'Ссылка': vacancy_url
        })

# Закрываем браузер
driver.quit()

# Сохраняем всё
with open('vacancies.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)
